In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import numpy as np
import concurrent.futures
import threading
from time import sleep

get links from meghdad

In [162]:
def get_soup(link,params=None):
    meghdad_header={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0',
    'Accept-Language':'en-US,en;q=0.5',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8'
    }
    result=requests.get(link,headers=meghdad_header,params=params).content
    soup = BeautifulSoup(result, 'html.parser')
    return soup
    

In [163]:
def get_number_of_pages(page_soup):
    return int(page_soup.select('#SharedMessage_ContentPlaceHolder1_rptPagingBottom_hlkPage_6')[0].text)


In [164]:
def find_laptop_links(page_number,lock,all_links):
    base_url='https://meghdadit.com/productlist/laptop/'
    page_soup=get_soup(base_url,params={'page':str(page_number)})
    laptops_li_tag=page_soup.find_all('li',class_='col-12 col-xl-3 d-flex list-item')
    this_page_info=[]
    for tag in laptops_li_tag:
            try:
                price_original=tag.find('span',class_='text-gray text-fa-h4 final-price position-relative').text.strip()
            except:
                price_original=None
            try:
                #so there was a mistake here, this tag(list_proce_of) is the original
                price_after_discount=tag.find('span',class_='list-price-off').text.strip()
            except:
                price_after_discount=None
            try:
                url=tag.find('a',class_='mh-100').get('href')
            except:
                url=None
            this_page_info.append([url,price_original,price_after_discount])
    with lock:
         all_links.extend(this_page_info)
    # with lock:
    #     links_list=page_soup.select('#SharedMessage_ContentPlaceHolder1_divThumbnailView >li>div >div >a.mh-100')
    #     lowest_prices=[span.text for span in page_soup.find_all('span',class_='text-gray text-fa-h4 final-price position-relative')]
    #     original_price=[span.text for span in page_soup.find_all('span',class_='list-price-off')]
    #     print(page_number)
    #     all_links.extend(list(zip([link.get('href') for link in links_list],original_price,lowest_prices)))
    return all_links
        

link='https://meghdadit.com/productlist/laptop'
first_page_soup=get_soup(link)
number_of_pages=get_number_of_pages(first_page_soup)
lock = threading.Lock()
all_links=[]
max_threads=20
with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
     futures = [executor.submit(find_laptop_links, page, lock, all_links) for page in np.arange(1, number_of_pages + 1)]
results = [future.result() for future in futures]

In [168]:
pd.DataFrame(all_links,columns=['url','price_after_discount','original_price']).to_csv('laptop_url_price.csv',index=False)

make data frames for laptops

In [169]:
original_df=pd.read_csv('laptop_url_price.csv',index_col=False)

In [187]:
# row=original_df[original_df.url=='/product/134403/lenovo-ideapad-3-core-i3-1115g4-20gb-512gb-ssd-intel-fhd-laptop/']
# # print(row)
# print(row['original_price'][0])
# print(row['price_after_discount'][0])

nan
17,270,000 تومان


In [233]:
laptop_urls=pd.read_csv('laptop_url_price.csv',index_col=False)['url'].to_list()
laptop_urls=['https://meghdadit.com'+x for x in laptop_urls[:]]

In [234]:
lock_index = threading.Lock()
df=pd.DataFrame()
max_threads=20
index=0

In [235]:
def build_df(url):
    soup=get_soup(url)
    features=soup.select('#specs div.attribute-caption')
    values=soup.select('#specs div.attribute-value')
    features_list=[x.text.strip() for x in features]
    values_list=[x.text.strip() for x in values]
    persian_info=soup.select('#SharedMessage_ContentPlaceHolder1_lblItemTitle')[0].text.strip()
    full_info=soup.select('#SharedMessage_ContentPlaceHolder1_lblItemTitle2')[0].text.strip()
    row=original_df[original_df.url==url.replace('https://meghdadit.com','')]
    # print(row)
    original_price=row['original_price'].values[0]
    after_discount=row['price_after_discount'].values[0]
    features_list.extend(['persian_info','full_info','original_price','after_discount'])
    values_list.extend([persian_info,full_info,original_price,after_discount])
    # print(features_list)
    # print(values_list)
    with lock_index: 
        global index  
        df.at[index,features_list]=values_list
        index+=1
        
with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
    executor.map(build_df,laptop_urls)

                                                 url price_after_discount  \
3  /product/125332/apple-macbook-air-2022-midnigh...     73,000,000 تومان   

     original_price  
3  74,000,000 تومان  
                                                 url price_after_discount  \
4  /product/125333/apple-macbook-air-2022-starlig...     73,300,000 تومان   

     original_price  
4  74,300,000 تومان  
                                                 url price_after_discount  \
2  /product/125331/apple-macbook-air-2022-silver-...     61,300,000 تومان   

     original_price  
2  62,300,000 تومان  
                                                 url price_after_discount  \
5  /product/125334/apple-macbook-air-2022-space-g...     74,300,000 تومان   

     original_price  
5  75,300,000 تومان  
                                                 url price_after_discount  \
0  /product/134403/lenovo-ideapad-3-core-i3-1115g...     17,270,000 تومان   

  original_price  
0            NaN  
           

In [236]:
df

,ابعاد,وزن,نوع دستگاه,اندازه صفحه نمایش,نوع صفحه نمایش,رزولوشن صفحه نمایش,صفحه نمایش لمسی,توضیحات صفحه نمایش,سازنده پردازنده,سری پردازنده,...,سایر امکانات,فرکانس رم,درگاه USB 3.2,کل حافظه گرافیکی,اقلام همراه لپ تاپ,درگاه mini Display,صفحه نمایش براق,سرعت چرخش هارد,معایب,اتصال NFC
0,30.41x21.24x1.11 سانتی ‌متر,1.24 کیلوگرم,مک‌بوک,13.6 اینچ,13.6inch (diagonal) LED-backlit display with I...,1600 * 2560,,500nits brightnessWide color (P3)True Tone tec...,Apple,M2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30.41x21.24x1.11 سانتی ‌متر,1.24 کیلوگرم,مک‌بوک,13.6 اینچ,13.6inch (diagonal) LED-backlit display with I...,1600 * 2560,,500nits brightnessWide color (P3)True Tone tec...,Apple,M2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30.41x21.24x1.11 سانتی ‌متر,1.24 کیلوگرم,مک‌بوک,13.6 اینچ,13.6inch (diagonal) LED-backlit display with I...,1600 * 2560,,500nits brightnessWide color (P3)True Tone tec...,Apple,M2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30.41x21.24x1.11 سانتی ‌متر,1.24 کیلوگرم,مک‌بوک,13.6 اینچ,13.6inch (diagonal) LED-backlit display with I...,1600 * 2560,,500nits brightnessWide color (P3)True Tone tec...,Apple,M2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,362.2×253.4×19.9 میلی‌متر,1.75 کلیوگرم,نوت بوک (لپ تاپ),15.6 اینچ,FHD TN,1080 * 1920,,NaN,INTEL,Core i3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6537,24 * 254 * 384 میلیمتر,2.19 کیلوگرم,نوت بوک (لپ تاپ),15.6 اینچ,TFT LED-backlit LCD,768 * 1366,,HD BrightView LED-backlit,INTEL,Core i7,...,NaN,NaN,NaN,4 گیگا بایت,NaN,NaN,NaN,5400 دور بر دقیقه,زاویه دید محدود صفحه‌ی نمایش,NaN
6538,10.4* 229 * 325 ميليمتر,1.11 کیلوگرم,آلترابوک,13.3 اینچ,IPS LCD,1080 * 1920,,صفحه نمايش 13.3 اينچي با روكش Corning Gorilla ...,INTEL,Core i5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6539,25 * 261 * 384 میلیمتر,2.29 کیلوگرم,نوت بوک (لپ تاپ),15.6 اینچ,TFT LED-backlit LCD,768 * 1366,,NaN,INTEL,Core i5,...,NaN,1600 مگا هرتز,NaN,NaN,NaN,NaN,NaN,5400 دور بر دقیقه,NaN,NaN
6540,24 * 241 * 345 میلیمتر,1.94 کیلوگرم,نوت بوک (لپ تاپ),14 اینچ,TFT LED-backlit LCD,1080 * 1920,,NaN,INTEL,Core i3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5400 دور بر دقیقه,NaN,NaN


In [237]:
df.to_csv('df.csv',index=False,encoding='utf-8')

In [8]:
# def get_json_file(content):
#     return json.loads(content)

In [9]:
# def get_page_info(page_num):
#     torob_header={
#     'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0',
#     'Accept-Language':'en-US,en;q=0.5',
#     'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8'
#         }
#     url_params={'page':str(page_num),'sort':'popularity','size':'24','category_name':'لپ-تاپ-و-نوت-بوک-laptop',
#                 'category_id':'99','category':'99','source':'next_desktop','_bt__experiment':'',
#                 'suid':'652bcbf4605b92d449536e70'}
#     link='https://api.torob.com/v4/base-product/search/'
#     page_content=requests.get(headers=torob_header,url=link,params=url_params).content
#     return get_json_file(page_content)


In [10]:
# page_num=0
# data=get_page_info(page_num)
# all_links=[]
# while True:
#     all_links.extend(['https://torob.com'+x['web_client_absolute_url'] for x in data['results']])
#     page_num+=1
#     data=get_page_info(page_num)
#     if data['next'] is None:
#        break
# all_links_df=pd.DataFrame(all_links,columns=['laptop_url'])
# all_links_df.to_csv('laptop_url.csv',index=False)